In [6]:
%%capture
%pip install - r requirement.txt
# python==3.6.13

In [7]:
import numpy as np
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMaximize, LpMinimize, LpBinary, LpStatus, value, PULP_CBC_CMD

debug = 1  # debug mode
TIMEOUT = 100 # timeout 
if debug:
    pd.set_option('display.max_columns', 500)  # show all columns
# randomly choose n samples
n = 30000

### PARAMETER INPUT

`None` stands for no boundary.

Hereby its just a demo.

In [8]:
# boolen
NbvCost = True  # True: max nbv; False: min cost

# float
minTotalNbv = 40000000
maxTotalNbv = 100000000

# float
minTotalCost = 20000000
maxTotalCost = 100000000

# float
fleetAgeLowBound = [None, 3, 4]
fleetAgeUpBound = [3, 6, 10]
fleetAgeLimit = [0.2, 0.3, 0.3]
fleetAgeGeq = [True, True, True] # True: >=, False: <=

# float
OnHireLimit = 1.0

# float
weightedAgeLowBound = [None, 3, 4]
weightedAgeUpBound = [4, 6, 15]
weightedAgeLimit = [0.4, 0.2, 0.3]
weightedAgeGeq = [True, True, True]

# string
productType = ['D20', 'D4H', 'R4H']
# float
productLimit = [0.1, 0.6, 0.0]
productGeq = [True, True, True]

# string
lesseeType = ['MSC', 'ONE', 'HAPAG']
# float
lesseeLimit = [0.7, 0.3, 0.2]
lesseeGeq = [False, False, False]

# string
contractType = ['LT', 'LE', 'LF']
# float
contractLimit = [0.3, 0.11, 0.15]
contractGeq = [True, True, True]

### READ DATA
TODO: read data

In [9]:
rawData = pd.read_excel(io='./FCI ANZ (2022-07-08) (NBV as at 30 Jun 2022)_v2.xlsx', sheet_name='Raw (portfolio)', engine='openpyxl')
data = rawData.iloc[:n, :61]
print(data.shape)

(30000, 61)


In [10]:
if debug:
    print(data['Product'].value_counts())
    print(data['Billing Status Fz'].value_counts())
    print(data['Contract Cust Id'].value_counts())
    print(data['Contract Lease Type'].value_counts())

D4H    23593
D20     6407
Name: Product, dtype: int64
ON    30000
Name: Billing Status Fz, dtype: int64
MSC         12618
ONE         11029
HAPAG        5778
TCLC          200
COSMR         149
CMA           126
PANOCEAN      100
Name: Contract Cust Id, dtype: int64
LT    16692
LE     7530
LF     5778
Name: Contract Lease Type, dtype: int64


### DATA PREPARATION

TODO: change the column name 

#### Container Age

Assign to `FleetAge1`, `FleetAge2`, `FleetAge3`

In [11]:
def SelectFleetAge(age, i):
    # check input
    if fleetAgeLowBound[i] is None:
        fleetAgeLowBound[i] = -float('inf')
    if fleetAgeUpBound[i] is None:
        fleetAgeUpBound[i] = float('inf')
        
    if fleetAgeLowBound[i] <= age < fleetAgeUpBound[i]:
        age = 1
    else:
        age = 0
    return age

data['FleetAge1'] = data.apply(lambda x: SelectFleetAge(x['Fleet Year Fz'], 0), axis=1)
data['FleetAge2'] = data.apply(lambda x: SelectFleetAge(x['Fleet Year Fz'], 1), axis=1)
data['FleetAge3'] = data.apply(lambda x: SelectFleetAge(x['Fleet Year Fz'], 2), axis=1)

#### Status

Assign new column `Status`

In [12]:
def SelectStatus(status):
    # no need to check input
    if status == 'ON':
        status = 1
    else:
        status = 0
    return status

data['Status'] = data.apply(lambda x: SelectStatus(x['Billing Status Fz']), axis=1)

#### Weighted Age

Assign new column `WeightedAge1`, `WeightedAge2`, `WeightedAge3`

In [13]:
def SelectWeightedAge(age, i):
    # check input
    if weightedAgeLowBound[i] is None:
        weightedAgeLowBound[i] = -float('inf')
    if weightedAgeUpBound[i] is None:
        weightedAgeUpBound[i] = float('inf')
        
    if weightedAgeLowBound[i] <= age < weightedAgeUpBound[i]:
        age = 1
    else:
        age = 0
    return age

data['WeightedAge1'] = data.apply(lambda x: SelectWeightedAge(x['Age x CEU'], 0), axis=1)
data['WeightedAge2'] = data.apply(lambda x: SelectWeightedAge(x['Age x CEU'], 1), axis=1)
data['WeightedAge3'] = data.apply(lambda x: SelectWeightedAge(x['Age x CEU'], 2), axis=1)

#### Product

Assign new column `ProductType1`, `ProductType2`, `ProductType3`

In [14]:
def SelectProductType(product, i):
    if product == productType[i]:
        product = 1
    else:
        product = 0
    return product

data['ProductType1'] = data.apply(lambda x: SelectProductType(x['Product'], 0), axis=1)
data['ProductType2'] = data.apply(lambda x: SelectProductType(x['Product'], 1), axis=1)
data['ProductType3'] = data.apply(lambda x: SelectProductType(x['Product'], 2), axis=1)

#### Lessee

Assign new column `Lessee1`, `Lessee2`, `Lessee3`

In [15]:
def SelectLessee(lessee, i):
    if lessee == lesseeType[i]:
        lessee = 1
    else:
        lessee = 0
    return lessee

data['Lessee1'] = data.apply(lambda x: SelectLessee(x['Contract Cust Id'], 0), axis=1)
data['Lessee2'] = data.apply(lambda x: SelectLessee(x['Contract Cust Id'], 1), axis=1)
data['Lessee3'] = data.apply(lambda x: SelectLessee(x['Contract Cust Id'], 2), axis=1)

#### Contract Type

Assign new column `ContractType1`, `ContractType2`, `ContractType3`

In [16]:
def SelectContractType(contract, i):
    if contract == contractType[i]:
        contract = 1
    else:
        contract = 0
    return contract

data['ContractType1'] = data.apply(lambda x: SelectContractType(x['Contract Lease Type'], 0), axis=1)
data['ContractType2'] = data.apply(lambda x: SelectContractType(x['Contract Lease Type'], 1), axis=1)
data['ContractType3'] = data.apply(lambda x: SelectContractType(x['Contract Lease Type'], 2), axis=1)

### SAVE DATA

In [17]:
if debug:
    data.to_csv('prepared_data_demo.csv')

In [18]:
if debug:
    data = pd.read_csv('./prepared_data_demo.csv')
data.sample(3)

,Unnamed: 0,Business Unit (Contract),Business Unit Am,Unit Id Fz,Cost,Po Num Fz,Mfr Dt Fz,Asset Id,Pool Fz,Product,Birth Unit Id Fz,Owner,Teu Fz,Ceu Fz,Contract Cust Id,Contract Num,Contract Dt,Contract Expiration Dt,Contract Lease Type,Contract Currency,Exchange Rate,Per Diem Rate (Max),Per Diem Rate (Max) in USD,Contract Rate Type (Min),Per Diem Rate (Min),Per Diem Rate (Min) in USD,Buy Out Amt Fz,Monthly Depr,Depre adjustment,Nbv,Life Status Fz,Billing Status Fz,Move Status Fz,Move Type Fz,Inv Status Fz,Fleet Year Fz,Financing Cd,Asset Status,Lessor,Remaining Lease Term,Age x CEU,RML x CEU,Mfr Year,Reserve Code Fz,Remaining Lease Term(C),RML x CEU(C),Chk on RML validity(HAPAG),Expiry year,Expiry month,Chk on RML validity(12 YR),Planned,Entity,Correct CEU,NP year,NBV check,age at drawdown X CEU,RML at drawdown X CEU,category,string,filter 2017 contracts,contract units,filter no. of units,FleetAge1,FleetAge2,FleetAge3,Status,WeightedAge1,WeightedAge2,WeightedAge3,ProductType1,ProductType2,ProductType3,Lessee1,Lessee2,Lessee3,ContractType1,ContractType2,ContractType3
28853,28853,FZOPS,DF3RM,FBIU0105714,2182.0,DFIC-FL-1801,1/2/2018,1080755.0,DFC3,D20,FBIU0105714,Dong Fang,1.0,1.0,HAPAG,LF-HAPAG-09,2017-11-01,2030-01-31,LF,USD,1.0,0.65,0.65,FR,0.35,0.35,0.0,-7.76,0.629,1784.485,AC,ON,GO,RLI,AVA,4.49,NaN,I,FUSBI,7.597260,4.490,7.597260,2018.0,F - OL / < 8 yrs,7.510000,7.510000,-0.08726,2030.0,1.0,12.000000,CNIC,FCI,1.0,2018 NP,Reasonable,5.240685,6.759315,from available portfolio,LF-HAPAG-09D20,NaN,1494.0,1.0,0,1,1,1,0,1,1,1,0,0,0,0,1,0,0,1
3116,3116,FZOPS,DF3RM,FDCU0056680,3682.0,DFIC-FL-1801,2/28/2018,1122352.0,DFC3,D4H,FDCU0056680,Dong Fang,2.0,1.7,MSC,LT-MSC-35B,2018-03-01,2032-07-31,LT,USD,1.0,1.07,1.07,ST,0.39,0.39,0.0,-15.42,0.681,2917.310,AC,ON,GO,RLI,AVA,4.33,NaN,I,FUSBI,10.095890,7.361,17.163014,2018.0,F - OL / < 8 yrs,10.095890,17.163014,0.00000,2032.0,7.0,14.425890,FCI ANZ 100M,FCI,1.7,2018 NP,Reasonable,8.637164,15.886849,from ANZ portfolio,LT-MSC-35BD4H,NaN,5621.0,1.0,0,1,1,1,0,0,1,0,1,0,1,0,0,1,0,0
9766,9766,FZOPS,DF3RM,FFAU3675741,4420.0,DFIC-FL-2104,3/24/2021,1700121.0,DFC3,D4H,FFAU3675741,Dong Fang,2.0,1.7,ONE,LE-ONE-09A,2020-11-16,2029-05-15,LT,USD,1.0,1.25,1.25,ST,0.63,0.63,0.0,-18.91,0.000,4136.340,AC,ON,GO,RLI,AVA,1.27,NaN,I,FUSBI,6.882192,2.159,11.699726,2021.0,F - OL / < 8 yrs,6.882192,11.699726,0.00000,2029.0,5.0,8.152192,FCI ANZ 100M,FCI,1.7,2021 NP,Reasonable,3.435164,10.423562,from ANZ portfolio,LE-ONE-09AD4H,NaN,2000.0,1.0,1,0,0,1,1,0,0,0,1,0,0,1,0,1,0,0


### Model Part

#### Prepare Parameters

convert to numpy

In [19]:
nbv = data['Nbv'].to_numpy()
cost = data['Cost'].to_numpy()
fleetAge = np.stack([data['FleetAge1'].to_numpy(),
                     data['FleetAge2'].to_numpy(),
                     data['FleetAge3'].to_numpy()])
status = data['Status'].to_numpy()
weightedAge = np.stack([data['WeightedAge1'].to_numpy(),
                        data['WeightedAge2'].to_numpy(),
                        data['WeightedAge3'].to_numpy()])
product = np.stack([data['ProductType1'].to_numpy(),
                        data['ProductType2'].to_numpy(),
                        data['ProductType3'].to_numpy()])
lessee = np.stack([data['Lessee1'].to_numpy(),
                   data['Lessee2'].to_numpy(),
                   data['Lessee3'].to_numpy()])
contract = np.stack([data['ContractType1'].to_numpy(),
                         data['ContractType2'].to_numpy(),
                         data['ContractType3'].to_numpy(),])
if debug:
    print('nbv shape: ', nbv.shape)
    print('cost shape:', cost.shape)
    print('fleetAge shape: ', fleetAge.shape)
    print('status shape:', status.shape)
    print('weightedAge shape:', weightedAge.shape)
    print('productType shape:', product.shape)
    print('lessee shape:', lessee.shape)
    print('contractType shape:', contract.shape)

nbv shape:  (30000,)
cost shape: (30000,)
fleetAge shape:  (3, 30000)
status shape: (30000,)
weightedAge shape: (3, 30000)
productType shape: (3, 30000)
lessee shape: (3, 30000)
contractType shape: (3, 30000)


#### Constraints

TODO: confirm constraints

`x`: np.array, size=num_of_container. `x[i]=1` stands for $i^{th}$ container is selected; `x[i]=0` stands for $i^{th}$ container is not selected.

1. minTotalNbv <= Total NBV <= maxTotalNbv
    
2. minTotalCost <= Total Cost <= maxTotalCost
    
3. Container Age >=
    
4. Status:

    1. OnHire >= OnHireLimit

5. Weighted Age >=

6. Product >=
    
7. Lessee <=
    
8. Contract Type >= 


#### Objective

1. min Cost: `Vars @ cost`

2. max Nbv: `Vars @ nbv`

#### Define Problem

In [20]:
# variables
var = np.array([LpVariable('container_{0}'.format(i), lowBound=0, cat=LpBinary) for i in range(n)])
# problem
prob = LpProblem("MyProblem", LpMaximize if NbvCost else LpMinimize)

In [21]:
# objective function 
if NbvCost:
    prob += lpSum(var * nbv)
else:
    prob += lpSum(var * cost)

In [22]:
# constraints
numSelected = lpSum(var) # num of selected containers

# nbv
if maxTotalNbv is not None:
    prob += lpSum(var * nbv) <= maxTotalNbv
if minTotalNbv is not None:
    prob += lpSum(var * nbv) >= minTotalNbv
    
# cost
if maxTotalCost is not None:
    prob += lpSum(var * cost) <= maxTotalCost
if minTotalCost is not None:
    prob += lpSum(var * cost) >= minTotalCost
    
# container age
if fleetAgeLimit[0] is not None:
    if fleetAgeGeq[0]:
        prob += lpSum(var * fleetAge[0]) >= fleetAgeLimit[0] * numSelected
    else:
        prob += lpSum(var * fleetAge[0]) <= fleetAgeLimit[0] * numSelected
if fleetAgeLimit[1] is not None:
    if fleetAgeGeq[1]:
        prob += lpSum(var * fleetAge[1]) >= fleetAgeLimit[1] * numSelected
    else:
        prob += lpSum(var * fleetAge[1]) <= fleetAgeLimit[1] * numSelected
if fleetAgeLimit[2] is not None:
    if fleetAgeGeq[2]:
        prob += lpSum(var * fleetAge[2]) >= fleetAgeLimit[2] * numSelected
    else:
        prob += lpSum(var * fleetAge[2]) <= fleetAgeLimit[2] * numSelected
    
# status
if OnHireLimit is not None:
    prob += lpSum(var * status) >= OnHireLimit * numSelected
    
# weighted age
if weightedAgeLimit[0] is not None:
    if weightedAgeGeq[0]:
        prob += lpSum(var * weightedAge[0]) >= weightedAgeLimit[0] * numSelected
    else:
        prob += lpSum(var * weightedAge[0]) <= weightedAgeLimit[0] * numSelected
if weightedAgeLimit[1] is not None:
    if weightedAgeGeq[1]:
        prob += lpSum(var * weightedAge[1]) >= weightedAgeLimit[1] * numSelected
    else:
        prob += lpSum(var * weightedAge[1]) <= weightedAgeLimit[1] * numSelected
if weightedAgeLimit[2] is not None:
    if weightedAgeGeq[2]:
        prob += lpSum(var * weightedAge[2]) >= weightedAgeLimit[2] * numSelected
    else:
        prob += lpSum(var * weightedAge[2]) <= weightedAgeLimit[2] * numSelected
        
# product
if productLimit[0] is not None:
    if productGeq[0]:
        prob += lpSum(var * product[0]) >= productLimit[0] * numSelected
    else:
        prob += lpSum(var * product[0]) <= productLimit[0] * numSelected
if productLimit[1] is not None:
    if productGeq[1]:
        prob += lpSum(var * product[1]) >= productLimit[1] * numSelected
    else:
        prob += lpSum(var * product[1]) <= productLimit[1] * numSelected
if productLimit[2] is not None:
    if productGeq[2]:
        prob += lpSum(var * product[2]) >= productLimit[2] * numSelected
    else:
        prob += lpSum(var * product[2]) <= productLimit[2] * numSelected

# lessee
if lesseeLimit[0] is not None:
    if lesseeGeq[0]:
        prob += lpSum(var * lessee[0]) >= lesseeLimit[0] * numSelected
    else:
        prob += lpSum(var * lessee[0]) <= lesseeLimit[0] * numSelected
if lesseeLimit[1] is not None:
    if lesseeGeq[1]:
        prob += lpSum(var * lessee[1]) >= lesseeLimit[1] * numSelected
    else:
        prob += lpSum(var * lessee[1]) <= lesseeLimit[1] * numSelected
if lesseeLimit[2] is not None:
    if lesseeGeq[2]:
        prob += lpSum(var * lessee[2]) >= lesseeLimit[2] * numSelected
    else:
        prob += lpSum(var * lessee[2]) <= lesseeLimit[2] * numSelected

# contract type
if contractLimit[0] is not None:
    if contractGeq[0]:
        prob += lpSum(var * contract[0]) >= contractLimit[0] * numSelected
    else:
        prob += lpSum(var * contract[0]) <= contractLimit[0] * numSelected
if contractLimit[1] is not None:
    if contractGeq[1]:
        prob += lpSum(var * contract[1]) >= contractLimit[1] * numSelected
    else:
        prob += lpSum(var * contract[1]) <= contractLimit[1] * numSelected
if contractLimit[2] is not None:
    if contractGeq[2]:
        prob += lpSum(var * contract[2]) >= contractLimit[2] * numSelected
    else:
        prob += lpSum(var * contract[2]) <= contractLimit[2] * numSelected

# prob.writeLP('problem.lp')

In [23]:
solver = PULP_CBC_CMD(msg = True, timeLimit=TIMEOUT)
prob.solve(solver)

1

In [24]:
print("==============================================================")
# print(prob)
print("status:",LpStatus[prob.status])
print("==============================================================")
print("target value: ",value(prob.objective))

status: Optimal
target value:  87703207.85900661


In [25]:
# if solution is found
if prob.status == 1 or prob.status == 2:
    result = np.array([var[i].varValue for i in range(n)])
    print(int(sum(result)), '/', n, 'containers are selected.')

24337 / 30000 containers are selected.


### Analysis

For debug only

TODO: if infeasible, it may take time much longer than expected

In [26]:
if debug:
    print('======================================================================')
    print("nbv:", round(sum(result * nbv), 2), 'constraints:', minTotalNbv, maxTotalNbv)
    print("cost:", round(sum(result * cost), 2), 'constraints:', minTotalCost, maxTotalCost)

    print("container age: ")
    print("\t container age from {0} to {1}:".format(fleetAgeLowBound[0], fleetAgeUpBound[0]), round(sum(result * fleetAge[0])/sum(result), 2), 'constraints:', fleetAgeLimit[0])
    print("\t container age from {0} to {1}:".format(fleetAgeLowBound[1], fleetAgeUpBound[1]), round(sum(result * fleetAge[1])/sum(result), 2), 'constraints:', fleetAgeLimit[1])
    print("\t container age from {0} to {1}:".format(fleetAgeLowBound[2], fleetAgeUpBound[2]), round(sum(result * fleetAge[2])/sum(result), 2), 'constraints:', fleetAgeLimit[2])

    print('billing status:', sum(result * status)/sum(result), 'constraints:', OnHireLimit)

    print("weighted age: ")
    print("\t weighted age from {0} to {1}:".format(weightedAgeLowBound[0], weightedAgeUpBound[0]), round(sum(result * weightedAge[0])/sum(result), 2), 'constraints:', weightedAgeLimit[0])
    print("\t weighted age from {0} to {1}:".format(weightedAgeLowBound[1], weightedAgeUpBound[1]), round(sum(result * weightedAge[1])/sum(result), 2), 'constraints:', weightedAgeLimit[1])
    print("\t weighted age from {0} to {1}:".format(weightedAgeLowBound[2], weightedAgeUpBound[2]), round(sum(result * weightedAge[2])/sum(result), 2), 'constraints:', weightedAgeLimit[2])
    
    print("product: ")
    print("\t product {0}:".format(productType[0]), round(sum(result * product[0])/sum(result), 2), 'constraints:', productLimit[0])
    print("\t product {0}:".format(productType[1]), round(sum(result * product[1])/sum(result), 2), 'constraints:', productLimit[1])
    print("\t product {0}:".format(productType[2]), round(sum(result * product[2])/sum(result), 2), 'constraints:', productLimit[2])
    
    print("lessee: ")
    print("\t lessee {0}:".format(lesseeType[0]), round(sum(result * lessee[0])/sum(result), 2), 'constraints:', lesseeLimit[0])
    print("\t lessee {0}:".format(lesseeType[1]), round(sum(result * lessee[1])/sum(result), 2), 'constraints:', lesseeLimit[1])
    print("\t lessee {0}:".format(lesseeType[2]), round(sum(result * lessee[2])/sum(result), 2), 'constraints:', lesseeLimit[2])
    
    print("contract type: ")
    print("\t contract type {0}:".format(contractType[0]), round(sum(result * contract[0])/sum(result), 2), 'constraints:', contractLimit[0])
    print("\t contract type {0}:".format(contractType[1]), round(sum(result * contract[1])/sum(result), 2), 'constraints:', contractLimit[1])
    print("\t contract type {0}:".format(contractType[2]), round(sum(result * contract[2])/sum(result), 2), 'constraints:', contractLimit[2])
    

nbv: 87703207.86 constraints: 40000000 100000000
cost: 99999998.07 constraints: 20000000 100000000
container age: 
	 container age from -inf to 3: 0.43 constraints: 0.2
	 container age from 3 to 6: 0.57 constraints: 0.3
	 container age from 4 to 10: 0.56 constraints: 0.3
billing status: 1.0 constraints: 1.0
weighted age: 
	 weighted age from -inf to 4: 0.43 constraints: 0.4
	 weighted age from 3 to 6: 0.21 constraints: 0.2
	 weighted age from 4 to 15: 0.57 constraints: 0.3
product: 
	 product D20: 0.21 constraints: 0.1
	 product D4H: 0.79 constraints: 0.6
	 product R4H: 0.0 constraints: 0.0
lessee: 
	 lessee MSC: 0.52 constraints: 0.7
	 lessee ONE: 0.3 constraints: 0.3
	 lessee HAPAG: 0.16 constraints: 0.2
contract type: 
	 contract type LT: 0.69 constraints: 0.3
	 contract type LE: 0.16 constraints: 0.11
	 contract type LF: 0.16 constraints: 0.15


### Output

In [27]:
if prob.status == 1 or prob.status == 2:
    data.drop(['FleetAge1', 'FleetAge2', 'FleetAge3', 'Status', 'WeightedAge1', 'WeightedAge2', 'WeightedAge3', 'ProductType1', 'ProductType2', 'ProductType3', 'Lessee1', 'Lessee2', 'Lessee3', 'ContractType1', 'ContractType2', 'ContractType3'], axis=1, inplace=True)
    data.insert(loc=0, column="selected", value=result)
    data.to_csv('demo_result.csv')

In [28]:
if prob.status == 1 and debug:
    data.sample(3)